# Data Cleaning and EDA

In [48]:
#| include: false

%matplotlib inline

import pandas as pd
from datetime import datetime
import numpy as np

from IPython.display import display, Latex, Markdown

import plotly.express as px
import plotly.io as pio
# Settings for plot rendering, makes work with HTML output + jupyer lab + static output
# pio.renderers.default = "notebook+plotly_mimetype+png"
pio.renderers.default = "plotly_mimetype+notebook_connected+png"

from ast import literal_eval

In [81]:
df_scraped = pd.read_csv("../data/scraped/scraped1_raw.csv", index_col=0)
df_scraped

,type,position,asin,name,image,has_prime,is_best_seller,is_amazon_choice,is_limited_deal,stars,total_reviews,url,availability_quantity,spec,price_string,price_symbol,price,original_price,section_name
0,search_product,1,B0B8GTTK6V,AppleWatch Series 7 [GPS + Cellular 45mm] Smar...,https://m.media-amazon.com/images/I/71zksDvWH-...,False,False,False,False,5.0,1.0,https://www.amazon.com/Apple-Cellular-Graphite...,1.0,{},$878.00,$,878.00,NaN,NaN
1,search_product,2,B0CSVG1668,AppleWatch Ultra 2 [GPS + Cellular 49mm] Smart...,https://m.media-amazon.com/images/I/81k6R2uh1J...,True,False,False,False,NaN,NaN,https://www.amazon.com/Apple-Cellular-Smartwat...,2.0,{},$799.00,$,799.00,NaN,NaN
2,search_product,3,B09HF4LPR5,AppleWatch Series 7 [GPS + Cellular 45mm] Smar...,https://m.media-amazon.com/images/I/91Kd2PB5Qs...,True,False,False,False,4.6,433.0,https://www.amazon.com/Apple-Cellular-Graphite...,1.0,{},$799.00,$,799.00,NaN,NaN
3,search_product,4,B0BDHT39HM,AppleWatch Ultra [GPS + Cellular 49mm] Smart w...,https://m.media-amazon.com/images/I/91jzT-Hi-T...,True,False,False,False,4.3,32.0,https://www.amazon.com/Apple-Cellular-Titanium...,1.0,{},$799.00,$,799.00,NaN,NaN
4,search_product,5,B0BHGJLX6W,AppleWatch Series 8 [GPS + Cellular 45mm] Smar...,https://m.media-amazon.com/images/I/61lCtY1nWP...,False,False,False,False,4.6,95.0,https://www.amazon.com/Apple-Cellular-Graphite...,12.0,{},$791.98,$,791.98,"{'price_string': '$828.00', 'price_symbol': '$...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24964,search_product,12,B0CV7DF1MC,"PUSOKEI Portable Monitor, 11.6 Inch 1080P FHD ...",https://m.media-amazon.com/images/I/51u58YcflH...,False,False,False,False,NaN,NaN,https://www.amazon.com/PUSOKEI-Portable-Monito...,NaN,{},$154.89,$,154.89,NaN,NaN
24965,search_product,13,B0CV7ZTT8Q,"PUSOKEI Portable Monitor, 13.3 Inch 1080P FHD ...",https://m.media-amazon.com/images/I/616U44i2ha...,False,False,False,False,NaN,NaN,https://www.amazon.com/PUSOKEI-Portable-Monito...,NaN,{},$162.19,$,162.19,NaN,NaN
24966,search_product,14,B0CV7XC894,Yoidesu 10.5in 1920x1280 Full HD Portable Gami...,https://m.media-amazon.com/images/I/61qU8np2VC...,False,False,False,False,NaN,NaN,https://www.amazon.com/Yoidesu-1920x1280-Porta...,NaN,{},$109.19,$,109.19,NaN,NaN
24967,search_product,15,B0CSFW92Q3,FEELWORLD DH156 15.6inch Portable Monitor 1080...,https://m.media-amazon.com/images/I/61orjMrisB...,True,False,False,False,NaN,NaN,https://www.amazon.com/FEELWORLD-15-6inch-Port...,7.0,{},$169.99,$,169.99,NaN,NaN


In [82]:
# Add `date_scraped` column
df_scraped["date_scraped"] = datetime(2024, 2, 1)

# Remove unecessary columns: 'type', 'position', 'has_prime', 'is_amazon_choice', 'is_limited_deal', 'availability_quantity', 'spec', 'price_string', 'price_symbol', 'section_name'
dropcols = ['type', 'position', 'has_prime', 'is_amazon_choice', 'is_limited_deal', 'availability_quantity', 'spec', 'price_string', 'price_symbol', 'section_name']
df_scraped = df_scraped.drop(dropcols, axis="columns", errors="ignore")

# Expand and fix original price
df_scraped["list_price"] = pd.json_normalize(df_scraped["original_price"].map(literal_eval, na_action="ignore"))["price"]
df_scraped["list_price"] = df_scraped["list_price"].fillna(df_scraped["price"])
df_scraped = df_scraped.drop("original_price", axis="columns", errors="ignore")

# Rename columns to match standard snake case for merging both datasets
df_scraped = df_scraped.rename(columns={"image": "image_url", "total_reviews": "reviews"})

# Drop rows with no asin or name or price
df_scraped = df_scraped.dropna(subset=["asin", "name", "price"])

# Fill NaN `reviews` column with 0
df_scraped["reviews"] = df_scraped["reviews"].fillna(0)

# Save cleaned df
df_scraped.to_csv("../data/scraped/scraped1.csv", index=False)

df_scraped

,asin,name,image_url,is_best_seller,stars,reviews,url,price,date_scraped,list_price
0,B0B8GTTK6V,AppleWatch Series 7 [GPS + Cellular 45mm] Smar...,https://m.media-amazon.com/images/I/71zksDvWH-...,False,5.0,1.0,https://www.amazon.com/Apple-Cellular-Graphite...,878.00,2024-02-01,878.00
1,B0CSVG1668,AppleWatch Ultra 2 [GPS + Cellular 49mm] Smart...,https://m.media-amazon.com/images/I/81k6R2uh1J...,False,NaN,0.0,https://www.amazon.com/Apple-Cellular-Smartwat...,799.00,2024-02-01,799.00
2,B09HF4LPR5,AppleWatch Series 7 [GPS + Cellular 45mm] Smar...,https://m.media-amazon.com/images/I/91Kd2PB5Qs...,False,4.6,433.0,https://www.amazon.com/Apple-Cellular-Graphite...,799.00,2024-02-01,799.00
3,B0BDHT39HM,AppleWatch Ultra [GPS + Cellular 49mm] Smart w...,https://m.media-amazon.com/images/I/91jzT-Hi-T...,False,4.3,32.0,https://www.amazon.com/Apple-Cellular-Titanium...,799.00,2024-02-01,799.00
4,B0BHGJLX6W,AppleWatch Series 8 [GPS + Cellular 45mm] Smar...,https://m.media-amazon.com/images/I/61lCtY1nWP...,False,4.6,95.0,https://www.amazon.com/Apple-Cellular-Graphite...,791.98,2024-02-01,828.00
...,...,...,...,...,...,...,...,...,...,...
24964,B0CV7DF1MC,"PUSOKEI Portable Monitor, 11.6 Inch 1080P FHD ...",https://m.media-amazon.com/images/I/51u58YcflH...,False,NaN,0.0,https://www.amazon.com/PUSOKEI-Portable-Monito...,154.89,2024-02-01,154.89
24965,B0CV7ZTT8Q,"PUSOKEI Portable Monitor, 13.3 Inch 1080P FHD ...",https://m.media-amazon.com/images/I/616U44i2ha...,False,NaN,0.0,https://www.amazon.com/PUSOKEI-Portable-Monito...,162.19,2024-02-01,162.19
24966,B0CV7XC894,Yoidesu 10.5in 1920x1280 Full HD Portable Gami...,https://m.media-amazon.com/images/I/61qU8np2VC...,False,NaN,0.0,https://www.amazon.com/Yoidesu-1920x1280-Porta...,109.19,2024-02-01,109.19
24967,B0CSFW92Q3,FEELWORLD DH156 15.6inch Portable Monitor 1080...,https://m.media-amazon.com/images/I/61orjMrisB...,False,NaN,0.0,https://www.amazon.com/FEELWORLD-15-6inch-Port...,169.99,2024-02-01,169.99


In [83]:
df_scraped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21517 entries, 0 to 24968
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   asin            21517 non-null  object        
 1   name            21517 non-null  object        
 2   image_url       21517 non-null  object        
 3   is_best_seller  21517 non-null  bool          
 4   stars           14525 non-null  float64       
 5   reviews         21517 non-null  float64       
 6   url             21517 non-null  object        
 7   price           21517 non-null  float64       
 8   date_scraped    21517 non-null  datetime64[ns]
 9   list_price      21517 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(4), object(4)
memory usage: 1.7+ MB


In [85]:
df_ksept = pd.read_csv("../data/kaggle_asaniczka/amazon_products.csv")
df_ksept_cats = pd.read_csv("../data/kaggle_asaniczka/amazon_categories.csv")

In [30]:
df_ksept

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,0.00,104,False,2000
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,104,False,1000
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.6,0,365.49,429.99,104,False,300
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.6,0,291.59,354.37,104,False,400
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.5,0,174.99,309.99,104,False,400
...,...,...,...,...,...,...,...,...,...,...,...
1426332,B00R3LIKCO,American Flag Patriotic USA Classic 5 Panel Me...,https://m.media-amazon.com/images/I/71PDJFz6AA...,https://www.amazon.com/dp/B00R3LIKCO,4.2,0,14.95,0.00,112,False,0
1426333,B098BQ7ZQ3,Men's Baseball Cap - H2O-DRI Line Up Curved Br...,https://m.media-amazon.com/images/I/812Tycexs4...,https://www.amazon.com/dp/B098BQ7ZQ3,4.4,0,33.99,0.00,112,False,0
1426334,B07X1MVNT1,[4 Pack] Adjustable Eyeglasses and Sunglasses ...,https://m.media-amazon.com/images/I/61vvYW1S9J...,https://www.amazon.com/dp/B07X1MVNT1,3.6,0,8.54,0.00,112,False,0
1426335,B08XLBG8V9,Ax2002 Aviator Sunglasses,https://m.media-amazon.com/images/I/51+yjD4F1x...,https://www.amazon.com/dp/B08XLBG8V9,4.5,0,54.36,57.39,112,False,0


In [87]:
# Add `date_scraped` column
df_ksept["date_scraped"] = datetime(2023, 11, 1)

# Remove unecessary columns `boughtInLastMonth`
df_ksept = df_ksept.drop("boughtInLastMonth", axis="columns", errors="ignore")

# Drop rows with any NaNs
df_ksept = df_ksept.dropna(subset=["title"])

# Fix list_price 0 to be instead equal to `price`
df_ksept["listPrice"] = df_ksept["listPrice"].replace(0.0, np.NaN).fillna(df_ksept["price"])

# Change category_id to actual category by using category table
df_ksept["category"] = df_ksept["category_id"].replace(to_replace=df_ksept_cats["id"].tolist(), value=df_ksept_cats["category_name"].tolist()).astype("category")
df_ksept = df_ksept.drop("category_id", axis="columns")

# Rename columns to match standard snake case for merging both datasets
df_ksept = df_ksept.rename(columns={"title": "name", "imgUrl": "image_url", "productURL": "url", "listPrice": "list_price", "isBestSeller": "is_best_seller"})

df_ksept

,asin,name,image_url,url,stars,reviews,price,list_price,is_best_seller,date_scraped,category
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,139.99,False,2023-11-01,Suitcases
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,False,2023-11-01,Suitcases
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.6,0,365.49,429.99,False,2023-11-01,Suitcases
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.6,0,291.59,354.37,False,2023-11-01,Suitcases
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.5,0,174.99,309.99,False,2023-11-01,Suitcases
...,...,...,...,...,...,...,...,...,...,...,...
1426332,B00R3LIKCO,American Flag Patriotic USA Classic 5 Panel Me...,https://m.media-amazon.com/images/I/71PDJFz6AA...,https://www.amazon.com/dp/B00R3LIKCO,4.2,0,14.95,14.95,False,2023-11-01,Men's Accessories
1426333,B098BQ7ZQ3,Men's Baseball Cap - H2O-DRI Line Up Curved Br...,https://m.media-amazon.com/images/I/812Tycexs4...,https://www.amazon.com/dp/B098BQ7ZQ3,4.4,0,33.99,33.99,False,2023-11-01,Men's Accessories
1426334,B07X1MVNT1,[4 Pack] Adjustable Eyeglasses and Sunglasses ...,https://m.media-amazon.com/images/I/61vvYW1S9J...,https://www.amazon.com/dp/B07X1MVNT1,3.6,0,8.54,8.54,False,2023-11-01,Men's Accessories
1426335,B08XLBG8V9,Ax2002 Aviator Sunglasses,https://m.media-amazon.com/images/I/51+yjD4F1x...,https://www.amazon.com/dp/B08XLBG8V9,4.5,0,54.36,57.39,False,2023-11-01,Men's Accessories


In [88]:
df_ksept.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1426336 entries, 0 to 1426336
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   asin            1426336 non-null  object        
 1   name            1426336 non-null  object        
 2   image_url       1426336 non-null  object        
 3   url             1426336 non-null  object        
 4   stars           1426336 non-null  float64       
 5   reviews         1426336 non-null  int64         
 6   price           1426336 non-null  float64       
 7   list_price      1426336 non-null  float64       
 8   is_best_seller  1426336 non-null  bool          
 9   date_scraped    1426336 non-null  datetime64[ns]
 10  category        1426336 non-null  category      
dtypes: bool(1), category(1), datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 112.9+ MB


In [89]:
# Save cleaned df
df_ksept.to_csv("../data/kaggle_asaniczka/amazon_products_cleaned.csv", index=False)

In [102]:
df_az = pd.concat([df_scraped, df_ksept], ignore_index=True)

# Remove duplicates (by asin + date scraped)
df_az = df_az.drop_duplicates(subset=["asin", "date_scraped"])

# Rename columns
df_az.columns = df_az.columns.map(lambda x : x.replace("_", " ")).str.title()

df_az

,Asin,Name,Image Url,Is Best Seller,Stars,Reviews,Url,Price,Date Scraped,List Price,Category
0,B0B8GTTK6V,AppleWatch Series 7 [GPS + Cellular 45mm] Smar...,https://m.media-amazon.com/images/I/71zksDvWH-...,False,5.0,1.0,https://www.amazon.com/Apple-Cellular-Graphite...,878.00,2024-02-01,878.00,NaN
1,B0CSVG1668,AppleWatch Ultra 2 [GPS + Cellular 49mm] Smart...,https://m.media-amazon.com/images/I/81k6R2uh1J...,False,NaN,0.0,https://www.amazon.com/Apple-Cellular-Smartwat...,799.00,2024-02-01,799.00,NaN
2,B09HF4LPR5,AppleWatch Series 7 [GPS + Cellular 45mm] Smar...,https://m.media-amazon.com/images/I/91Kd2PB5Qs...,False,4.6,433.0,https://www.amazon.com/Apple-Cellular-Graphite...,799.00,2024-02-01,799.00,NaN
3,B0BDHT39HM,AppleWatch Ultra [GPS + Cellular 49mm] Smart w...,https://m.media-amazon.com/images/I/91jzT-Hi-T...,False,4.3,32.0,https://www.amazon.com/Apple-Cellular-Titanium...,799.00,2024-02-01,799.00,NaN
4,B0BHGJLX6W,AppleWatch Series 8 [GPS + Cellular 45mm] Smar...,https://m.media-amazon.com/images/I/61lCtY1nWP...,False,4.6,95.0,https://www.amazon.com/Apple-Cellular-Graphite...,791.98,2024-02-01,828.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1447848,B00R3LIKCO,American Flag Patriotic USA Classic 5 Panel Me...,https://m.media-amazon.com/images/I/71PDJFz6AA...,False,4.2,0.0,https://www.amazon.com/dp/B00R3LIKCO,14.95,2023-11-01,14.95,Men's Accessories
1447849,B098BQ7ZQ3,Men's Baseball Cap - H2O-DRI Line Up Curved Br...,https://m.media-amazon.com/images/I/812Tycexs4...,False,4.4,0.0,https://www.amazon.com/dp/B098BQ7ZQ3,33.99,2023-11-01,33.99,Men's Accessories
1447850,B07X1MVNT1,[4 Pack] Adjustable Eyeglasses and Sunglasses ...,https://m.media-amazon.com/images/I/61vvYW1S9J...,False,3.6,0.0,https://www.amazon.com/dp/B07X1MVNT1,8.54,2023-11-01,8.54,Men's Accessories
1447851,B08XLBG8V9,Ax2002 Aviator Sunglasses,https://m.media-amazon.com/images/I/51+yjD4F1x...,False,4.5,0.0,https://www.amazon.com/dp/B08XLBG8V9,54.36,2023-11-01,57.39,Men's Accessories


In [103]:
# Save cleaned df
df_az.to_csv("../data/products_cleaned.csv", index=False)

In [104]:
df_az[df_az["Asin"].duplicated(keep=False)].sort_values(by="Asin")

,Asin,Name,Image Url,Is Best Seller,Stars,Reviews,Url,Price,Date Scraped,List Price,Category
20414,B00003IEM9,Energizer MAX Alkaline Batteries AAA 2 Pack,https://m.media-amazon.com/images/I/81ZAKiZk7h...,False,4.7,845.0,https://www.amazon.com/Energizer-MAX-Alkaline-...,3.25,2024-02-01,3.25,NaN
718579,B00003IEM9,Energizer MAX Alkaline Batteries AAA 2 Pack,https://m.media-amazon.com/images/I/81ZAKiZk7h...,False,4.7,0.0,https://www.amazon.com/dp/B00003IEM9,3.27,2023-11-01,3.27,Household Supplies
556358,B00005IBX9,Cuisinart DCC-1200P1 Brew Central 12-Cup Progr...,https://m.media-amazon.com/images/I/61-qv4WD-j...,False,4.6,0.0,https://www.amazon.com/dp/B00005IBX9,97.00,2023-11-01,97.00,Home Appliances
6340,B00005IBX9,Cuisinart DCC-1200P1 Brew Central 12-Cup Progr...,https://m.media-amazon.com/images/I/61sZMasBgR...,False,4.6,12339.0,https://www.amazon.com/Cuisinart-DCC-1200-Cent...,99.95,2024-02-01,99.95,NaN
20456,B00008XPN0,"Energizer MAX AAA Batteries (12 Pack), Triple ...",https://m.media-amazon.com/images/I/71NKUau16V...,False,4.8,7185.0,https://www.amazon.com/Energizer-Max-Alkaline-...,9.45,2024-02-01,13.99,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15448,B0CJCHDKLF,"Wireless Earbuds Bluetooth Headphones, 40H Pla...",https://m.media-amazon.com/images/I/61IOFpeimn...,False,4.8,2624.0,https://www.amazon.com/YAQ-Bluetooth-Headphone...,21.99,2024-02-01,79.99,NaN
222493,B0CJKVT223,"MSI Pro MP251, 25"", IPS, 1920 x 1080 (FHD), 10...",https://m.media-amazon.com/images/I/81vm74fzYN...,False,0.0,0.0,https://www.amazon.com/dp/B0CJKVT223,119.99,2023-11-01,119.99,Computer Monitors
11922,B0CJKVT223,MSI PRO MP251 24.5-inch IPS 1920 x 1080 (FHD) ...,https://m.media-amazon.com/images/I/81vm74fzYN...,False,4.5,261.0,https://www.amazon.com/MSI-MP251-24-5-inch-Ada...,79.99,2024-02-01,119.99,NaN
949581,BT00CTP2GM,Amazon.com Gift Card in a Greeting Card (Vario...,https://m.media-amazon.com/images/I/71AnOa2lFQ...,False,4.8,0.0,https://www.amazon.com/dp/BT00CTP2GM,15.00,2023-11-01,15.00,Gift Cards
